Resources Used
- wget.download('https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py')
- Setup https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

# 0. Setup Paths

In [1]:
# 17:00
WORKSPACE_PATH = 'Tensorflow/workspace' # Main workspace 
SCRIPTS_PATH = 'Tensorflow/scripts' # 
APIMODEL_PATH = 'Tensorflow/models' # TF onject detection model library from Github
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations' # label map will go to this file
IMAGE_PATH = WORKSPACE_PATH+'/images' # labeling and images will stay
MODEL_PATH = WORKSPACE_PATH+'/models' # Trained Model will stay here. when yu train this model, you will get number of checkpoints that you can then restore from. in order to build real-time model. real-time detection 19:00
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models' # leveraging pre-trained models 
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config' # when we are using the object . this gives you all the details foer the deep learing model that you use.
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/' #checkpoint paths inside of it. config path is pointing directly to pipeline.config file

# 1. Create Label Map

In [2]:
# 19:00
# "Labels Dictionary"
labels = [{'name':'Mask', 'id':1}, {'name':'NoMask', 'id':2}]

# output this into a "label map". when working with the TF object detection library this needs to be in the format of pb text(protobuff file). this is just what the object detection library likes to use.
with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f: # go into annotation_path, and then create a new file called labelmap.pbtxt. and set that to "w(write)" as you will be writing in to it. 
    for label in labels: #then we are looping through each one of these lables as below.. in order to work with out bbtxt file we writing these out into a specific format.
        f.write('item { \n') # writing "item" line
        f.write('\tname:\'{}\'\n'.format(label['name'])) # writing "name" line
        f.write('\tid:{}\n'.format(label['id'])) # writing "id" line
        f.write('}\n') # final line is closing "}" it out

# 2. Create TF records

In [3]:
#23:00 
# leveraging the script you got in scripts folder "generate_tfrecord". "generate_tfrecord" is from the official object detection api tutorial https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html . this "generate_tfrecord" file is very critical as it makes sure that you get your dat in the right format. in order to run this tf record or generate tf record script, we are just going to be executing a command inside of your notebook. below are the command
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'} # use python command, and then you go into your scripts path, and then forward slash generate tfrecord.py so this is basically allowing you to pas variables from your notebook to your command line so your scripts path which is basically from 'Tensorflow/scripts' (as shows in 0. Setup Paths). so this Tensorflow/scripts/generate_tfrecord.py'. then you do a similar thing for all of the other compomnenst. so then pass through image path. First, '/tain' path, and passing through '/label_map.bptxt', and then specify what you want the name ofthe output to be. this case, you specify "-o" passing through annotation path and then specifying '/train.'. this is for your training images, you also need to do it for your testing images as below. so you copy and paste and replace '/train' to '/test' ,as well as  '/train.' to '/test.record'
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [4]:
# commandr to be triggered is first up going to cd into tensorflow dir so this is to clone it into your tensorflow dir to keep everything nice and neat. and then you will be cloning this repo here so if you actually open this up. this is basically the tensorflow model garden. and its also got all of this good stuff (in the repo). you actually need to go train a model. so you can just go and trigger that command and this is going to go and clone that repo so this might takea little bit of time because theres actually quite much to be loaded
!cd Tensorflow && git clone https://github.com/tensorflow/models

Cloning into 'models'...


In [6]:
#wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
#!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
#!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# 4. Copy Model Config to Training Folder

In [7]:
# set up Model Config. whenever you are training using the obhjecty detection api ypu basically build a model from a model config. this case, it is preloaded in the models folder.
# /Users/yuyaokamura/py-object-detection/Tensorflow/workspace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8 
# "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8" is from "TensorFlow model zoo"
#if you want to use different model, you can definetely do that. Go to "https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md" or "https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md". choose a model of your choice(maybe better off picking one beased on speed especially for REAL-TIME Obejct detection) and replace. (direction is shown in latter half of part.3 28:30)

 # this is basically taken your pretrained config file from that pretrained model zoo model that you downloaded from the model zoo, and pasted it into here.
 #config this pipeline config to be specific to your model so youre goin to point it to you tf records and were also going to change a couple of other parameters depending on your hardware.

# Copy Model Config to Training Folder
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' # you create custom model name. feel free to change the mame.

In [47]:
# Copy Model Config to Training Folder pt.2
!mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME} # mkdir to create that directory first
!cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME} # then uyou used cp bash command to copy that pipeline config and in this case see this is where all of you pre-traine paths come in handy becuase you dont need to write them every single time. then you copied it into the "model path" ( as specified in pt.0) so "/models". then you paste it into your custom model folder"my_ssd_mobnet"

A subdirectory or file Tensorflow\workspace\models\my_ssd_mobnet already exists.


# 5. Update Config For Transfer Learning

In [1]:
# you first need to import 2 dependencies.
import tensorflow as tf#  -object detection utility. so including config util and pipeline pb2 so this will work with protobufs 
from object_detection.utils import config_util 
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format #  -text format from google protobuf.

In [8]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config' #

In [55]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH) #Grab config 

In [56]:
config

{'model': ssd {
   num_classes: 2
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 4e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.01
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.997
         scale: true
         epsilon: 0.001
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matcher {
     argmax_matcher {
       matched_threshold: 0.5
    

In [52]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [53]:
pipeline_config.model.ssd.num_classes = 2
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [54]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [58]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=5000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=5000


# 7. Load Train Model From Checkpoint

In [2]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [9]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 8. Detect in Real-Time

In [10]:
import cv2 
import numpy as np

In [11]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [105]:
cap.release()

In [12]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

In [42]:
detections = detect_fn(input_tensor)

In [67]:
from matplotlib import pyplot as plt